# Import dependencies

In [1]:
import pandas
import numpy as np
import mlflow
import tensorflow
from tensorflow import keras
import mlflow.keras
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.model_selection import train_test_split

# Retrieve Data

In [2]:
pandas_df = pandas.read_csv("training_data.csv")
X=pandas_df.iloc[:,:-1]
Y=pandas_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=4284, stratify=Y)

# Set Experiment

In [3]:

mlflow.set_experiment("Baseline_Predictions")
mlflow.tensorflow.autolog()

INFO: 'Baseline_Predictions' does not exist. Creating a new experiment


# Create Model

In [4]:

model = keras.Sequential([
  keras.layers.Dense(
    units=36,
    activation='relu',
    input_shape=(X_train.shape[-1],)
  ),
  keras.layers.BatchNormalization(),
  keras.layers.Dense(units=1, activation='sigmoid'),
])

model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001),
  loss="binary_crossentropy",
  metrics="Accuracy"
)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Run the Model

In [5]:
with mlflow.start_run(run_name='keras_model_baseline') as run:
    model.fit(
        X_train,
        y_train,
        epochs=20,
        validation_split=0.05,
        shuffle=True,
        verbose=0
    )
    preds = model.predict(X_test)
    y_pred = np.where(preds>0.5,1,0)
    f1 = f1_score(y_test, y_pred)
    mlflow.log_metric(key="f1_experiment_score", value=f1)


2021/07/02 19:27:52 INFO mlflow.utils.autologging_utils: tensorflow autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow to the MLflow run with ID '0fb5bdf74e564135bdd037b4c341aabd'


2021/07/02 19:27:57 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during tensorflow autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/tensorflow.py:771: UserWarning: Logging to MLflow failed: module 'google.protobuf.descriptor' has no attribute '_internal_create_key'"
